In [1]:
import pandas as pd
from geopy.distance import geodesic
from datetime import datetime
import gmplot
import folium
import numpy as np
#import time

# 兩點距離求速度_1

In [3]:
gps_data_df=pd.read_excel(r'./user_gps_data1.xlsx',encoding='utf-16',index_col=0)

localFormat = "%Y-%m-%d %H:%M:%S"
gps_data_df['point_speed']=0
gps_data_df['point_distance']=0
for user in list(set(gps_data_df['user'])):
    for i in gps_data_df[gps_data_df['user']==user].index:
        if i < gps_data_df[gps_data_df['user']==user].index[-1]:
            deltatime=datetime.strptime(gps_data_df.loc[i+1,'time'],localFormat)-datetime.strptime(gps_data_df.loc[i,'time'],localFormat)
            minutesdelta=(deltatime.seconds)/60
            if minutesdelta<10:
                prev=gps_data_df.loc[i,['latitude','longitude']]
                post=gps_data_df.loc[i+1,['latitude','longitude']]
                if minutesdelta>0:
                    gps_data_df.loc[i+1,'point_speed']=((geodesic(prev,post).kilometers)/minutesdelta)*60
                gps_data_df.loc[i+1,'point_distance']=(geodesic(prev,post).kilometers)
    #print(prev,post)
gps_data_df.head()

,user,time,latitude,longitude,elevation,speed,source,horizontal_dilution,vertical_dilution,course,satellites,point_speed,point_distance
0,user1,2019-11-26 16:44:00,25.052852,121.607082,42.199997,NaN,network,NaN,NaN,NaN,NaN,0.000000,0.000000
1,user1,2019-11-26 16:44:34,25.052853,121.607081,42.199997,NaN,network,NaN,NaN,NaN,NaN,0.013911,0.000131
2,user1,2019-11-26 16:45:46,25.052889,121.607220,35.699997,NaN,network,NaN,NaN,NaN,NaN,0.730402,0.014608
3,user1,2019-11-26 16:46:48,25.052797,121.607039,NaN,NaN,network,NaN,NaN,NaN,NaN,1.216668,0.020954
4,user1,2019-11-26 16:47:58,25.052446,121.607163,24.500000,NaN,network,NaN,NaN,NaN,NaN,2.102851,0.040889


In [5]:
gps_data_df[['user','time','latitude','longitude','point_speed']].head()

,user,time,latitude,longitude,point_speed
0,user1,2019-11-26 16:44:00,25.052852,121.607082,0.000000
1,user1,2019-11-26 16:44:34,25.052853,121.607081,0.013911
2,user1,2019-11-26 16:45:46,25.052889,121.607220,0.730402
3,user1,2019-11-26 16:46:48,25.052797,121.607039,1.216668
4,user1,2019-11-26 16:47:58,25.052446,121.607163,2.102851


# 求staypoint_2

In [12]:
#求staypoint
gps_data_df=pd.read_excel(r'./gps_data_pointspeed2.xlsx',encoding='utf-16',index_col=0)
#門檻值
#3km/hr 15mins or #1.5km/hr 15mins
speed_threshold=3
stay_point_user_list={}
gps_data_df['stay_point_lat']=0
gps_data_df['stay_point_long']=0
localFormat = "%Y-%m-%d %H:%M:%S"
for user in list(set(gps_data_df['user'])):    
    stay_point_coor_lat=[]
    stay_point_coor_lon=[]
    stay_time=0
    stay_point_list=[]
    stay_point_index=[]
    for i in gps_data_df[gps_data_df['user']==user].index[1:]:
                #if speed <3
                if gps_data_df.loc[i,'point_speed']<speed_threshold:
                    #設定每個點時間，加總每個點時間
                    #total_time+=time_delta
                    deltatime=datetime.strptime(gps_data_df.loc[i,'time'],localFormat)-datetime.strptime(gps_data_df.loc[i-1,'time'],localFormat)
                    minutesdelta=(deltatime.seconds)/60
                    if minutesdelta<15:
                        stay_time +=minutesdelta
                        #stay point.append(coord)
                        stay_point_coor_lat.append(gps_data_df.loc[i,'latitude'])
                        stay_point_coor_lon.append(gps_data_df.loc[i,'longitude'])
                        stay_point_index.append(i)
                    else:
                        if stay_point_coor_lat:
                            stay_point=(sum(stay_point_coor_lat)/len(stay_point_coor_lat),
                                        sum(stay_point_coor_lon)/len(stay_point_coor_lon))
                            gps_data_df.loc[stay_point_index,'stay_point_lat']=stay_point[0]
                            gps_data_df.loc[stay_point_index,'stay_point_long']=stay_point[1]
                        stay_point_list.append(stay_point)
                        stay_point_coor_lat=[]
                        stay_point_coor_lon=[]
                        stay_point_index=[]
                        stay_time=0
                        #stay point.append(coord)


                    #if total_time>10 minutes

                elif gps_data_df.loc[i,'point_speed'] > speed_threshold and stay_time>15:
                        stay_point=(sum(stay_point_coor_lat)/len(stay_point_coor_lat),
                                    sum(stay_point_coor_lon)/len(stay_point_coor_lon))
                        gps_data_df.loc[stay_point_index,'stay_point_lat']=stay_point[0]
                        gps_data_df.loc[stay_point_index,'stay_point_long']=stay_point[1]
                        stay_point_list.append(stay_point)
                        stay_point_coor_lat=[]
                        stay_point_coor_lon=[]
                        stay_point_index=[]
                        stay_time=0
                else:
                    stay_point_coor_lat=[]
                    stay_point_coor_lon=[]
                    stay_point_index=[]
                    stay_time=0
    stay_point_user_list[user]=stay_point_list
                
#print(stay_point_user_list)
                

In [13]:
gps_data_df.to_excel(r'./gps_data_staypoint.xlsx',encoding='utf-16')

# 每個位置合併成一個_3

In [14]:
#座標合併成1格 & 只取日期
stay_point_df=pd.read_excel(r'./gps_data_staypoint3.xlsx',encoding='utf-16',index_col=0)
subset = stay_point_df[['stay_point_lat','stay_point_long']]
subset
stay_point_df['date']=[x[:10] for x in stay_point_df['time'].values]
stay_point_df['stay_point_coord']=[tuple(x) for x in subset.to_numpy()]
#del stay_point_df['coord']

In [15]:
#求每點間隔時間
stay_point_df_d=stay_point_df

stay_point_df_d['delta_time']=0

stay_point_df_d.head()

localFormat = "%Y-%m-%d %H:%M:%S"
for user in list(set(stay_point_df_d['user'])):    
    for i in stay_point_df_d[stay_point_df_d['user']==user].index[1:]:
                    #設定每個點時間，加總每個點時間
                    #total_time+=time_delta
                    deltatime=datetime.strptime(stay_point_df_d.loc[i,'time'],localFormat)-datetime.strptime(stay_point_df_d.loc[i-1,'time'],localFormat)
                    minutesdelta=(deltatime.seconds)/60
                    if minutesdelta<15:
                        stay_point_df_d.loc[i,'delta_time']=minutesdelta
                        #print(stay_point_df_d.loc[i,'delta_time'])


In [16]:
stay_point_df_d.to_excel(r'./stay_point_df_complete.xlsx',encoding='utf=16')

## 求stay_point 開始及結束時間，group成1組_4

In [17]:
#求stay_point 開始及結束時間


stay_point_df_complete=pd.read_excel(r'./stay_point_df_complete4.xlsx',encoding='utf-16',index_col=0)
stay_point_group_df_time=stay_point_df_complete[['user','date','time','stay_point_coord']]
group_df=stay_point_group_df_time.groupby(['user','date','stay_point_coord'])
list(group_df)
group_df.agg([np.min,np.max])
#
localFormat = "%Y-%m-%d %H:%M:%S"
groupby_time=group_df.agg([np.min,np.max])
groupby_time.columns

tmp=[]
for x,n in zip(groupby_time['time']['amax'].values,groupby_time['time']['amin'].values):
    deltatime=(datetime.strptime(x,localFormat)-datetime.strptime(n,localFormat)).seconds/60
    tmp.append(deltatime)

groupby_time['duration']=pd.Series(tmp).values


In [ ]:
groupby_time.to_excel(r'./group_time.xlsx',encoding='utf-16')

In [ ]:
#求兩stay_point中間 最大速度及開始結束時間

In [6]:
#求起訖時間
stay_point_group_df_time=stay_point_df_complete[['user','date','time','stay_point_coord']]
group_df=stay_point_group_df_time.groupby(['user','date','stay_point_coord'])
list(group_df)
group_df.agg([np.min,np.max])

time  \
                                                                        amin   
user  date       stay_point_coord                                              
user1 2019-11-26 (0.0, 0.0)                              2019-11-26 16:44:00   
                 (24.95123293461539, 121.5527319884615)  2019-11-26 21:18:17   
                 (25.0420301197541, 121.5318861763114)   2019-11-26 17:14:53   
                 (25.04349100095238, 121.5326256890476)  2019-11-26 19:41:29   
      2019-11-28 (0.0, 0.0)                              2019-11-28 18:31:31   
      2019-12-06 (0.0, 0.0)                              2019-12-06 20:06:42   
                 (24.67489107, 121.7704191553846)        2019-12-06 21:09:14   
                 (24.6758851996875, 121.768344229375)    2019-12-06 21:28:57   
                 (24.6773619152381, 121.7710231332234)   2019-12-06 22:17:32   
                 (24.67737572222222, 121.7710075388889)  2019-12-06 20:22:12   
                 (24.6784706, 121.7707810588235)         2019-12-06 20:45:19   
      2019-12-07 (0.0, 0.0)                              2019-12-07 13:22:46   
                 (24.6773619152381, 121.7710231332234)   2019-12-07 00:00:44   
user2 2019-11-30 (0.0, 0.0)                              2019-11-30 11:14:49   
                 (25.05279420846153, 121.546710233077)   2019-11-30 17:19:45   
                 (25.05280191666667, 121.54428395)       2019-11-30 13:30:29   
                 (25.05280249375, 121.544260675)         2019-11-30 17:44:10   
                 (25.05280527, 121.5442672911111)        2019-11-30 11:26:55   
                 (25.0536779, 121.547662)                2019-11-30 15:13:34   
                 (25.0537584, 121.5477461)               2019-11-30 16:28:49   
      2019-12-06 (0.0, 0.0)                              2019-12-06 08:04:20   
                 (25.0496129, 121.37488)                 2019-12-06 09:18:13   
                 (25.050077925, 121.37560130625)         2019-12-06 11:58:22   
                 (25.05081852125, 121.3744898641666)     2019-12-06 14:50:40   
                 (25.05082983818182, 121.3744881509091)  2019-12-06 16:01:26   
                 (25.05083111, 121.3744936606141)        2019-12-06 09:19:24   
                 (25.05083348057692, 121.3744970475961)  2019-12-06 12:40:13   
                 (25.05083499444444, 121.3744953462963)  2019-12-06 08:05:55   
user3 2019-12-02 (0.0, 0.0)                              2019-12-02 19:32:31   
                 (25.04235834232283, 121.5785223655512)  2019-12-02 19:33:37   
...                                                                      ...   
user7 2019-11-27 (24.96409703928571, 121.1927717378571)  2019-11-27 05:00:54   
                 (24.96410330615385, 121.1927429851282)  2019-11-27 07:25:58   
                 (24.96415156430379, 121.1928223844304)  2019-11-27 01:22:37   
                 (24.96416655461538, 121.1928489761538)  2019-11-27 10:00:07   
                 (24.9641742, 121.192758)                2019-11-27 12:29:13   
                 (24.9642245275, 121.1926372075)         2019-11-27 07:01:16   
                 (24.9642528, 121.1929226)               2019-11-27 23:13:27   
                 (24.96780865, 121.1917346)              2019-11-27 14:48:03   
                 (24.9678255, 121.1917589)               2019-11-27 18:06:03   
      2019-11-29 (0.0, 0.0)                              2019-11-29 10:44:37   
                 (24.96773219166667, 121.1918529066667)  2019-11-29 16:33:40   
                 (24.96778933333333, 121.1915949333333)  2019-11-29 13:09:31   
                 (24.967798375, 121.191621975)           2019-11-29 10:53:25   
                 (24.9677999, 121.1917276)               2019-11-29 12:32:43   
                 (24.9678336, 121.19181905)              2019-11-29 15:03:44   
                 (25.042284822, 121.578467646)           2019-11-29 22:02:54   
                 (25.0423506, 121.5785174)               2019-11-29 22:3

In [2]:
groupby_time=pd.read_excel(r'./group_time.xlsx',encoding='utf-16')

In [7]:
#
localFormat = "%Y-%m-%d %H:%M:%S"
groupby_time=group_df.agg([np.min,np.max])
groupby_time.columns
#deltatime=datetime.strptime(groupby_time['time']['amax'].values,localFormat)-datetime.strptime(groupby_time['time']['amin'].values,localFormat)
#print(deltatime)
#minutesdelta=(deltatime.seconds)/60
tmp=[]
for x,n in zip(groupby_time['time']['amax'].values,groupby_time['time']['amin'].values):
    deltatime=(datetime.strptime(x,localFormat)-datetime.strptime(n,localFormat)).seconds/60
    tmp.append(deltatime)

groupby_time['duration']=pd.Series(tmp).values
    #groupby_time.loc[i,'duration']=
#-groupby_time['time']['amin']
#groupby_time['duration']=groupby_time['duration']

In [8]:

#.append(pd.Series(tmp),ignore_index=True)
#pd.Series(tmp)
#groupby_time.sort_values(by=['amin'])

In [85]:
stay_point_df_d=stay_point_df

# 求每點時間

In [65]:
groupby_time.to_excel(r'./group_time.xlsx',encoding='utf-16')

In [22]:
groupby_time

time  \
                                                                        amin   
user  date       stay_point_coord                                              
user1 2019-11-26 (0.0, 0.0)                              2019-11-26 16:44:00   
                 (24.95123293461539, 121.5527319884615)  2019-11-26 21:18:17   
                 (25.0420301197541, 121.5318861763114)   2019-11-26 17:14:53   
                 (25.04349100095238, 121.5326256890476)  2019-11-26 19:41:29   
      2019-11-28 (0.0, 0.0)                              2019-11-28 18:31:31   
      2019-12-06 (0.0, 0.0)                              2019-12-06 20:06:42   
                 (24.67489107, 121.7704191553846)        2019-12-06 21:09:14   
                 (24.6758851996875, 121.768344229375)    2019-12-06 21:28:57   
                 (24.6773619152381, 121.7710231332234)   2019-12-06 22:17:32   
                 (24.67737572222222, 121.7710075388889)  2019-12-06 20:22:12   
                 (24.6784706, 121.7707810588235)         2019-12-06 20:45:19   
      2019-12-07 (0.0, 0.0)                              2019-12-07 13:22:46   
                 (24.6773619152381, 121.7710231332234)   2019-12-07 00:00:44   
user2 2019-11-30 (0.0, 0.0)                              2019-11-30 11:14:49   
                 (25.05279420846153, 121.546710233077)   2019-11-30 17:19:45   
                 (25.05280191666667, 121.54428395)       2019-11-30 13:30:29   
                 (25.05280249375, 121.544260675)         2019-11-30 17:44:10   
                 (25.05280527, 121.5442672911111)        2019-11-30 11:26:55   
                 (25.0536779, 121.547662)                2019-11-30 15:13:34   
                 (25.0537584, 121.5477461)               2019-11-30 16:28:49   
      2019-12-06 (0.0, 0.0)                              2019-12-06 08:04:20   
                 (25.0496129, 121.37488)                 2019-12-06 09:18:13   
                 (25.050077925, 121.37560130625)         2019-12-06 11:58:22   
                 (25.05081852125, 121.3744898641666)     2019-12-06 14:50:40   
                 (25.05082983818182, 121.3744881509091)  2019-12-06 16:01:26   
                 (25.05083111, 121.3744936606141)        2019-12-06 09:19:24   
                 (25.05083348057692, 121.3744970475961)  2019-12-06 12:40:13   
                 (25.05083499444444, 121.3744953462963)  2019-12-06 08:05:55   
user3 2019-12-02 (0.0, 0.0)                              2019-12-02 19:32:31   
                 (25.04235834232283, 121.5785223655512)  2019-12-02 19:33:37   
...                                                                      ...   
user7 2019-11-27 (24.96409703928571, 121.1927717378571)  2019-11-27 05:00:54   
                 (24.96410330615385, 121.1927429851282)  2019-11-27 07:25:58   
                 (24.96415156430379, 121.1928223844304)  2019-11-27 01:22:37   
                 (24.96416655461538, 121.1928489761538)  2019-11-27 10:00:07   
                 (24.9641742, 121.192758)                2019-11-27 12:29:13   
                 (24.9642245275, 121.1926372075)         2019-11-27 07:01:16   
                 (24.9642528, 121.1929226)               2019-11-27 23:13:27   
                 (24.96780865, 121.1917346)              2019-11-27 14:48:03   
                 (24.9678255, 121.1917589)               2019-11-27 18:06:03   
      2019-11-29 (0.0, 0.0)                              2019-11-29 10:44:37   
                 (24.96773219166667, 121.1918529066667)  2019-11-29 16:33:40   
                 (24.96778933333333, 121.1915949333333)  2019-11-29 13:09:31   
                 (24.967798375, 121.191621975)           2019-11-29 10:53:25   
                 (24.9677999, 121.1917276)               2019-11-29 12:32:43   
                 (24.9678336, 121.19181905)              2019-11-29 15:03:44   
                 (25.042284822, 121.578467646)           2019-11-29 22:02:54   
                 (25.0423506, 121.5785174)               2019-11-29 22:3

# 相鄰stay_point距離小於50m合併_5

In [ ]:
user_stay_point_daily=pd.read_excel(r'./user_stay_point_daily5.xlsx',encoding='utf-16')
#調整stay_point位置
#若下個staypoint距離前個<30m  視同同一個staypoint
for u in set(user_stay_point_daily['user']):
    date_set=set(user_stay_point_daily[user_stay_point_daily['user']==u]['date'])
    user_index=user_stay_point_daily[user_stay_point_daily['user']==u].index
    user_df=user_stay_point_daily.loc[user_index]
    for d in date_set:
        user_date_index=user_df[user_df['date']==d].index
        for i in user_date_index:
            if i+1 in user_date_index:
                now=eval(user_stay_point_daily.loc[i,'stay_point_coord'])
                nextp=eval(user_stay_point_daily.loc[i+1,'stay_point_coord'])
                dist=geodesic(now,nextp).kilometers
                if dist<0.05:
                    user_stay_point_daily.loc[i+1,'stay_point_coord']=user_stay_point_daily.loc[i,'stay_point_coord']
    #for d in date_set:
       
       #user_stay_point_daily[user_stay_point_daily['user']==u and user_stay_point_daily['date']==d]
user_stay_point_daily    

In [ ]:
#groupby 開始時間及duration
user_stay_point_adjust=user_stay_point_daily
adjust_group=user_stay_point_adjust.groupby(['user','date','stay_point_coord']).agg({'start':'min',
                                                                       'end':'max',
                                                                       'duration':'sum'})

#break multiindex###########
adjust_group.reset_index(inplace=True)
#組內依starttime排序
adjust_group_2=adjust_group.groupby(['user','date']).apply(lambda x:x.sort_values('start'))

In [ ]:
adjust_group_2.to_excel(r'./stay_point_nearbygroup.xlsx',encoding='utf-16')

In [4]:
adjust_group=pd.read_excel(r'./stay_point_nearbygroup.xlsx',encoding='utf-16')

,user,date,stay_point_coord,start,end,duration
0,user1,2019-11-26,"(25.052463, 121.604865)",2019-11-26 13:18:17,2019-11-26 16:30:43,190.433333
1,user1,2019-11-26,"(25.0420301197541, 121.5318861763114)",2019-11-26 17:14:53,2019-11-26 19:39:05,144.200000
2,user1,2019-11-26,"(25.04349100095238, 121.5326256890476)",2019-11-26 19:41:29,2019-11-26 20:06:49,25.333333
3,user1,2019-11-26,"(24.95123293461539, 121.5527319884615)",2019-11-26 21:18:17,2019-11-26 21:47:43,29.433333
4,user1,2019-12-06,"(24.67737572222222, 121.7710075388889)",2019-12-06 20:22:12,2019-12-06 20:42:01,19.816667
5,user1,2019-12-06,"(24.6784706, 121.7707810588235)",2019-12-06 20:45:19,2019-12-06 21:03:04,17.750000
6,user1,2019-12-06,"(24.67489107, 121.7704191553846)",2019-12-06 21:09:14,2019-12-06 21:24:07,14.883333
7,user1,2019-12-06,"(24.6758851996875, 121.768344229375)",2019-12-06 21:28:57,2019-12-06 22:04:46,35.816667
8,user1,2019-12-06,"(24.6773619152381, 121.7710231332234)",2019-12-06 22:17:32,2019-12-07 13:16:47,897.783333
9,user2,2019-11-30,"(25.05280527, 121.5442672911111)",2019-11-30 11:26:55,2019-11-30 13:44:29,135.883333


In [10]:
#調整duration時間
localFormat = "%Y-%m-%d %H:%M:%S"
for i in adjust_group.index:
    start=adjust_group.loc[i,'start']
    end=adjust_group.loc[i,'end']
    deltatime=(datetime.strptime(str(end),localFormat)-datetime.strptime(str(start),localFormat)).seconds/60
    adjust_group.loc[i,'duration']=deltatime
adjust_group

,user,date,stay_point_coord,start,end,duration
0,user1,2019-11-26,"(25.052463, 121.604865)",2019-11-26 13:18:17,2019-11-26 16:30:43,192.433333
1,user1,2019-11-26,"(25.0420301197541, 121.5318861763114)",2019-11-26 17:14:53,2019-11-26 19:39:05,144.200000
2,user1,2019-11-26,"(25.04349100095238, 121.5326256890476)",2019-11-26 19:41:29,2019-11-26 20:06:49,25.333333
3,user1,2019-11-26,"(24.95123293461539, 121.5527319884615)",2019-11-26 21:18:17,2019-11-26 21:47:43,29.433333
4,user1,2019-12-06,"(24.67737572222222, 121.7710075388889)",2019-12-06 20:22:12,2019-12-06 20:42:01,19.816667
5,user1,2019-12-06,"(24.6784706, 121.7707810588235)",2019-12-06 20:45:19,2019-12-06 21:03:04,17.750000
6,user1,2019-12-06,"(24.67489107, 121.7704191553846)",2019-12-06 21:09:14,2019-12-06 21:24:07,14.883333
7,user1,2019-12-06,"(24.6758851996875, 121.768344229375)",2019-12-06 21:28:57,2019-12-06 22:04:46,35.816667
8,user1,2019-12-06,"(24.6773619152381, 121.7710231332234)",2019-12-06 22:17:32,2019-12-07 13:16:47,899.250000
9,user2,2019-11-30,"(25.05280527, 121.5442672911111)",2019-11-30 11:26:55,2019-11-30 13:44:29,137.566667


In [11]:
adjust_group.to_excel(r'./user_stay_point_final.xlsx',encoding='utf-16')

In [ ]:
# #求速度

# for user 
# for i in index
# speed_list=[]
# if stay_point ==(0,0) and point_speed>0:
#     #df.loc[i,'end_time']=df.loc[i,'time']
#     speed_list.append(df.loc[i,'point_speed'])    
# else:
#     if speed_list:
#         df.loc[i]['move_max_speed']=max(speed_list)
#         df.loc[i,'start_time']=df.loc[i,'time']
#         speed_list=[]
        


